## Download Twitter Data

<!--
import data_analytics.github as github
print(github.create_jupyter_notebook_header("markcrowe-com", "data-analytics-project-template", "notebooks/notebook-1-02-dc-twitter-api.ipynb", "master")
-->
<table style="margin: auto;"><tr><td><a href="https://mybinder.org/v2/gh/markcrowe-com/data-analytics-project-template/master?filepath=notebooks/notebook-1-02-dc-twitter-api.ipynb" target="_parent"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a></td><td>online editors</td><td><a href="https://colab.research.google.com/github/markcrowe-com/data-analytics-project-template/blob/master/notebooks/notebook-1-02-dc-twitter-api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></td></tr></table>

### Setup
Import required third party Python libraries, import supporting functions and sets up data source file paths.

In [1]:
# Local
#!pip install -r script/requirements.txt
# Remote option
#!pip install -r https://raw.githubusercontent.com/markcrowe-com/data-analytics-project-template/requirements.txt
#Options: --quiet --user

In [2]:
from configparser import ConfigParser
from pandas import DataFrame
import tweepy

### Twitter
#### Irish Farming Accounts

Dept of Agriculture, Food and the Marine:  <a href="https://twitter.com/agriculture_ie" target="_new">@agriculture_ie</a>  
Leading the sustainable development of Ireland’s agri-food, forestry and marine sectors.

See [config.ini.sample](config.ini.sample) for the ini file format.

In [3]:
config_filepath = "config.ini"
config_parser = ConfigParser()
config_parser.read(config_filepath)

access_token = config_parser["Twitter"]["AccessToken"]
access_token_secret = config_parser["Twitter"]["AccessTokenSecret"]
consumer_key = config_parser["Twitter"]["ApiKey"]
consumer_secret = config_parser["Twitter"]["ApiKeySecret"]

<https://python-twitter.readthedocs.io/en/latest/getting_started.html>  


You need to have a developer account: <https://developer.twitter.com/en/portal/petition/essential/basic-info>

And apply for elevated access.
<https://developer.twitter.com/en/portal/products/elevated>

In [4]:
o_auth_handler = tweepy.OAuthHandler(consumer_key, consumer_secret)
o_auth_handler.set_access_token(access_token, access_token_secret)
tweepy_api = tweepy.API(o_auth_handler, wait_on_rate_limit=True)

In [5]:
#Dept of Agriculture, Food and the Marine
screen_name = "agriculture_ie"

In [6]:
tweets = tweepy_api.user_timeline(
    screen_name=screen_name,
    count=200,  # 200 is the maximum allowed count
    include_rts=False,
    tweet_mode="extended"
)  # Necessary to keep full_text otherwise only the first 140 words are extracted

In [7]:
for tweet in tweets[:3]:
    print("ID: {}".format(tweet.id))
    print(tweet.created_at)
    print(tweet.full_text)
    print("\n")

ID: 1486384591313293320
2022-01-26 17:04:36+00:00
Reopening the scheme,Minister @pippa_hackett said with 'expanding market opportunities for Irish organic produce and the drive for Irish farming to be more environmentally friendly, now is surely the time for all Irish farmers to give serious consideration to joining the scheme.'


ID: 1486383042587418633
2022-01-26 16:58:27+00:00
Farmers could qualify for yearly payments of up to €220/ha during the conversion period and up to €170/ha for full organic status.

Higher rates are available for organic #horticultural and #tillage farmers

For a guide to the application process, visit: https://t.co/wPRP91ZYJU


ID: 1486383040310005765
2022-01-26 16:58:27+00:00
New applicants interested in converting to #organic farming can apply to our Organic Farming Scheme when it reopens on Wednesday, February 9 with the budget increased by a further €5 million for new entrants
🗓️Farmers have until Friday, April 8 to apply
📍https://t.co/Chd5TGz9uZ https:/

In [8]:
#extract additional tweets

all_tweets = []
all_tweets.extend(tweets)
oldest_id = tweets[-1].id
while True:
    tweets = tweepy_api.user_timeline(
        screen_name=screen_name,
        count=200,# 200 is the maximum allowed count
        include_rts=False,
        max_id=oldest_id - 1,
        # Necessary to keep full_text
        # otherwise only the first 140 words are extracted
        tweet_mode='extended')
    if len(tweets) == 0:
        break
    oldest_id = tweets[-1].id
    all_tweets.extend(tweets)
    print('N of tweets downloaded till now {}'.format(len(all_tweets)))

N of tweets downloaded till now 249
N of tweets downloaded till now 383
N of tweets downloaded till now 534
N of tweets downloaded till now 676
N of tweets downloaded till now 832
N of tweets downloaded till now 992
N of tweets downloaded till now 1170
N of tweets downloaded till now 1340
N of tweets downloaded till now 1478
N of tweets downloaded till now 1594
N of tweets downloaded till now 1715
N of tweets downloaded till now 1827
N of tweets downloaded till now 1934
N of tweets downloaded till now 2058
N of tweets downloaded till now 2172
N of tweets downloaded till now 2205


In [9]:
tweets_list: list = [[
    tweet.id_str, tweet.user.screen_name, tweet.created_at,
    tweet.favorite_count, tweet.retweet_count,
    tweet.full_text.encode("utf-8").decode("utf-8")
] for idx, tweet in enumerate(all_tweets)]

In [10]:
tweet_columns = [
    "id", "screen_name", "created_at", "favorite_count", "retweet_count",
    "text"
]
dataframe = DataFrame(tweets_list, columns=tweet_columns)
dataframe.to_csv('./../assets/twitter-agriculture-ie.csv', index=False)
dataframe.head(3)

,id,screen_name,created_at,favorite_count,retweet_count,text
0,1486384591313293320,agriculture_ie,2022-01-26 17:04:36+00:00,5,5,"Reopening the scheme,Minister @pippa_hackett s..."
1,1486383042587418633,agriculture_ie,2022-01-26 16:58:27+00:00,3,3,Farmers could qualify for yearly payments of u...
2,1486383040310005765,agriculture_ie,2022-01-26 16:58:27+00:00,24,30,New applicants interested in converting to #or...
